In [1]:
# 1. 检索相似问题
# 一个相似问题对应5个id

# 2. 根据相似问题path标签获取文本块
# 返回文本块


In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [3]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_path = r"D:/Python/models/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_path)

C:\Users\pixiu\AppData\Local\Temp\ipykernel_28568\534697663.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_path)
d:\Python\MiniConda\envs\itext2kg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.vectorstores import FAISS
import pickle

def loader_faiss(faiss_index_path, embeddings_model_path):
    """
    加载 FAISS 索引
    :param faiss_index_path: FAISS 索引路径
    :param embeddings_model_path: 嵌入模型路径
    :return: FAISS 向量数据库对象
    """
    # 加载 FAISS 索引
    index = faiss.read_index(faiss_index_path + 'faiss_index.index')
    with open(faiss_index_path + "docstore.pkl", "rb") as f:
        docstore = pickle.load(f)

    with open(faiss_index_path + "index_to_docstore_id.pkl", "rb") as f:
        index_to_docstore_id = pickle.load(f)

    # 从加载的索引创建 FAISS 向量数据库
    vector_store_loaded = FAISS(
        embedding_function=HuggingFaceEmbeddings(model_name=embeddings_model_path),
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )
    return vector_store_loaded

In [5]:
embeddings_model_path = r"D:/Python/models/bge-large-zh-v1.5"
q_index_path = 'D:\\code\\agent_demo\\kg\\make_index\\questions\\'
c_index_path = 'D:\\code\\agent_demo\\kg\\make_index\\book\\'
q_vector_store_loaded = loader_faiss(q_index_path, embeddings_model_path)
c_vector_store_loaded = loader_faiss(c_index_path, embeddings_model_path)

# 现在，你可以进行搜索
query = "学科知识是教师所应具备的专业知识的主体。"
search_results = q_vector_store_loaded.similarity_search(query, k=5)  # k 是返回的结果数量
# 打印搜索结果
for result in search_results:
    print(result)

page_content='在教师专业知识结构中，[[本体性知识/学科知识]]是指教师具有的特定的学科知识，是教师知识结构的核心。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196], [2005, 3015, 188, 190, 196, 198], [2005, 3015, 212, 213, 215, 216], [2005, 3015, 188, 190, 196, 200], [2005, 3015, 188, 190, 196, 197]]', 'answer': '["本体性知识\\/学科知识"]', 'analysis': nan}
page_content='[[学科教学知识]]又称为教学内容知识，这是教师专业知识的核心、重要而最有用的成分。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196, 199], [2005, 3015, 188, 190, 196, 198], [2005, 3015, 188, 190, 196, 197], [2005, 3015, 188, 190, 196], [2005, 3015, 212, 213, 215, 216]]', 'answer': '["学科教学知识"]', 'analysis': nan}
page_content='教师专业知识主要包括本体性知识、[[条件性知识]]、实践性知识和一般文化知识。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196], [2005, 3015, 212, 213, 215, 216], [2005, 3015, 188, 190, 196, 198], [2005, 

In [8]:
def get_concept_by_question(question, embedding_model_path, q_index_path, c_index_path, k=3):
    """
    根据问题获取概念
    :param question: 问题
    :param vector_store_loaded: 向量数据库对象
    :param k: 返回的结果数量
    :return: 概念列表
    """
    q_vector_store_loaded = loader_faiss(q_index_path, embeddings_model_path)
    c_vector_store_loaded = loader_faiss(c_index_path, embeddings_model_path)
    
    import json
    with open("neo4j_export.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    
    node_dict = {node["id"]: node for node in data["nodes"]}

    # 进行相似性搜索
    sim_q = q_vector_store_loaded.similarity_search(question, k=k)
    # sim_c = c_vector_store_loaded.similarity_search(question, k=k)

    source_list = []
    for q in sim_q:
        # 获取问题的ID
        import ast
        meatdata = q.metadata
        path = ast.literal_eval(meatdata["path"])
        for p in path:
            source_list.append(p)

    result_list = []
    from collections import Counter
    # 先把每个 list 转成 tuple，用来计数
    source_tuples = [tuple(x) for x in source_list]
    # 用 Counter 统计频率
    counter = Counter(source_tuples)
    # 按出现次数排序，并再转回 list
    sorted_unique_sources = [list(item) for item, count in counter.most_common()]
    for path in sorted_unique_sources:
        # 获取问题的ID
        source_id = path[-1]
        print(path)
        c_node = node_dict[source_id]
        
        node1 = node_dict[path[0]]
        node2 = node_dict[path[1]]
        path_str = '第' + str(node1['properties']['num']) + '章' + node1['properties']['title'] + '\n'
        path_str += '第' + str(node2['properties']['num']) + '节' + node2['properties']['title'] + '\n'
        for p in path[2:]:
            node = node_dict[p]
            path_str += '----' + node['properties']['name'] + '----\n'
        
        result_list.append((c_node['properties']['name'] + c_node['properties']['page_content'], path_str))

    return result_list


In [9]:
query = "学科知识是教师所应具备的专业知识的主体。"
result = get_concept_by_question(query, embeddings_model_path, q_index_path, c_index_path, k=3)
print(result)

[2005, 3015, 188, 190, 196]
[2005, 3015, 188, 190, 196, 198]
[2005, 3015, 212, 213, 215, 216]
[2005, 3015, 188, 190, 196, 200]
[2005, 3015, 188, 190, 196, 197]
[2005, 3015, 188, 190, 196, 199]
[2005, 3015, 212, 220, 221]
[('专业知识\n教师的专业知识是教师职业区别于其他职业的理论体系与经验系统。教师专业知识的质的深化，体现了教师职业的专业性，它是衡量教师专业化程度的标志之一。教师应具备以下几方面的知识。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师的专业素养----\n----教师专业素养的结构----\n----专业知识----\n'), ('学科知识\n学科知识是教师所应具备的专业知识的主体。教师要想顺利完成某一学科复杂的教学活动，必须对该学科的内容有深入透彻的了解，精通该学科的知识。教师不仅要理解和掌握所教学科的基本事实、概念、原理、理论和学科框架以及探究或思维的方式，还要了解和熟练掌握所教学科的最新发展以及与其他学科和社会实践的联系。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师的专业素养----\n----教师专业素养的结构----\n----专业知识----\n----学科知识----\n'), ('教育知识技能的体系化。即形成一套关于教育教学的专业知识技能体系，作为教师教育的内容和教师从事教育教学工作的依据。教师职业专业化的知识技能体系既包括学科专业知识技能，也包括教育专业知识技能。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师专业发展----\n----教师专业发展概述----\n----教师群体专业化----\n----教育知识技能的体系化----\n'), ('通识性知识\n通识性知识是教师应具备的基础知识，包括自然学科、人文社会学科、人文艺术学科以及适应教育内容、教学手段和方法现代化的信息技术知识。广博的文化修养具有陶冶人文精神、养成人文素养的内在价值。教师拥有广博的一般文化科学知识，能够适应学生的好奇心和求知欲，拓展学生的精神世界，发展学生智慧，从而使学生内